In [1]:
import _pickle as cPickle
import docplex
from docplex.mp.model import Model

In [2]:
with open(r"../Data/test_optimize.pickle", "rb") as input_file:
    df = cPickle.load(input_file)

with open(r"../Data/team.pickle", "rb") as input_file:
    team = cPickle.load(input_file)

In [3]:
team_dict = dict(zip(team.code.to_list(),team.short_name.to_list()))

In [4]:
df

,id,web_name,team,element_type,now_cost,predict_points,actual_points
0,1,Cédric,1,2,42,1.658569,2
1,3,Xhaka,1,3,51,59.257935,62
2,4,Elneny,1,3,42,6.226771,4
3,5,Holding,1,2,42,1.271105,4
4,6,Partey,1,3,48,40.345520,42
...,...,...,...,...,...,...,...
662,625,Diego Costa,20,4,55,10.143930,7
663,629,Traoré,20,3,45,10.171195,12
664,657,Fraser,20,4,45,0.082976,0
665,658,Griffiths,20,3,45,0.082976,0


In [5]:
!pip install pulp

In [6]:
import scipy
import numpy as np
from scipy.optimize import minimize
import pulp

In [7]:
def select_team(expected_scores, prices, positions, clubs, total_budget=100, sub_factor=0.2):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ] 


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

In [8]:
df.columns

Index(['id', 'web_name', 'team', 'element_type', 'now_cost', 'predict_points',
       'actual_points'],
      dtype='object')

In [9]:
expected_scores = df.predict_points
prices = df.now_cost / 10
position = df.element_type
team = df.team
names = df.web_name

decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, position.values, team.values)

/home/caleb/PythonEnvs/FPL/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/caleb/PythonEnvs/FPL/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/dcf3f45388fe493b8465d26ad47978d9-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/dcf3f45388fe493b8465d26ad47978d9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1373 COLUMNS
At line 16645 RHS
At line 18014 BOUNDS
At line 20016 ENDATA
Problem MODEL has 1368 rows, 2001 columns and 9268 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1079.94 - 0.00 seconds
Cgl0004I processed model has 1365 rows, 2001 columns (2001 integer (2001 of which binary)) and 8671 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.603175
Cbc0038I Solution found of -1075.99
Cbc0038I Before mini branch and bound, 1999 integers at bound fixed and 0 continuous
Cbc0038I Full problem 1365 rows 200

In [18]:
pos_map = {1:'GK', 2:'DEF', 3: 'MID', 4:'FWD'}
for i in range(df.shape[0]):
    if decisions[i].value() != 0:
        print(f'{names[i]} {pos_map[position[i]]}: {expected_scores[i]} points at ${prices[i]}')
print('subs')
for i in range(df.shape[0]):
    if sub_decisions[i].value() != 0:
        print(f'{names[i]} {pos_map[position[i]]}: {expected_scores[i]} points at ${prices[i]}')
#print('Capt')
#for i in range(df.shape[0]):
#    if captain_decisions[i].value != 0:
#        print(f'Captain: {names[i]} score: {captain_decisions[i].value}')

Ødegaard MID: 77.52822875976562 points at $6.4
Raya GK: 56.88882827758789 points at $4.6
Trossard MID: 82.91353607177734 points at $7.1
Maddison MID: 79.3056869506836 points at $8.2
Firmino FWD: 89.04657745361328 points at $8.1
Cancelo DEF: 65.36078643798828 points at $7.4
Haaland FWD: 119.04466247558594 points at $12.2
Trippier DEF: 90.45354461669922 points at $5.9
Schär DEF: 60.936946868896484 points at $4.9
Almirón MID: 91.85137176513672 points at $5.8
Kane FWD: 101.47588348388672 points at $11.6
subs
White DEF: 58.11398696899414 points at $4.7
Tarkowski DEF: 52.87712097167969 points at $4.4
Andreas MID: 59.64226531982422 points at $4.6
Ward GK: 53.81494903564453 points at $4.1
